In [2]:
import pandas as pd

In [3]:
df = pd.read_json("frames.json")

In [4]:
df.head()

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [5]:
import pandas as pd
import json

# df : ton DataFrame original

rows = []
for _, row in df.iterrows():
    conv_id = row["id"]
    user_id = row["user_id"]
    wizard_id = row["wizard_id"]
    for turn in row["turns"]:         # chaque élément est un dict
        turn_copy = turn.copy()
        turn_copy["conv_id"] = conv_id
        turn_copy["user_id"] = user_id
        turn_copy["wizard_id"] = wizard_id
        rows.append(turn_copy)

turns_df = pd.DataFrame(rows)


In [6]:
def turn_to_jsonl_line(turn_row):
    if turn_row.get("author") != "user":
        return None

    text = turn_row.get("text", "")
    frames = turn_row.get("labels", {}).get("frames", [])
    info = frames[0].get("info", {}) if frames else {}

    def get_val(key):
        arr = info.get(key, [])
        return arr[0]["val"] if arr else None

    intent = get_val("intent")
    or_city = get_val("or_city")
    dst_city = get_val("dst_city")
    budget = get_val("budget")

    output = {}
    if intent is not None:
        output["intent"] = intent
    if or_city is not None:
        output["or_city"] = or_city
    if dst_city is not None:
        output["dst_city"] = dst_city
    if budget is not None:
        try:
            output["budget"] = float(budget)
        except ValueError:
            output["budget"] = budget

    jsonl_obj = {
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are an assistant that extracts flight booking information: "
                    "intent, or_city, dst_city, dep_date, ret_date, budget. "
                    "Always answer with a valid JSON object and nothing else."
                )
            },
            {
                "role": "user",
                "content": text
            },
            {
                "role": "assistant",
                "content": json.dumps(output, ensure_ascii=False)
            },
        ]
    }
    return json.dumps(jsonl_obj, ensure_ascii=False)


In [7]:
# appliquer la fonction sur chaque ligne de turns_df
turns_df.head()

,text,labels,author,timestamp,conv_id,user_id,wizard_id,db
0,I'd like to book a trip to Atlantis from Capri...,"{'acts': [{'args': [{'val': 'book', 'key': 'in...",user,1.471272e+12,e2c0fc6c-2134-4891-8353-ef16d8412c9a,U22HTHYNP,U21DKG18C,NaN
1,"Hi...I checked a few options for you, and unfo...",{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471272e+12,e2c0fc6c-2134-4891-8353-ef16d8412c9a,U22HTHYNP,U21DKG18C,{'result': [[{'trip': {'returning': {'duration...
2,"Yes, how about going to Neverland from Caprica...","{'acts': [{'args': [{'val': 'Neverland', 'key'...",user,1.471273e+12,e2c0fc6c-2134-4891-8353-ef16d8412c9a,U22HTHYNP,U21DKG18C,NaN
3,I checked the availability for this date and t...,{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471273e+12,e2c0fc6c-2134-4891-8353-ef16d8412c9a,U22HTHYNP,U21DKG18C,"{'result': [[], [], [], [], [], []], 'search':..."
4,I have no flexibility for dates... but I can l...,"{'acts': [{'args': [{'val': False, 'key': 'fle...",user,1.471273e+12,e2c0fc6c-2134-4891-8353-ef16d8412c9a,U22HTHYNP,U21DKG18C,NaN


In [8]:
# appliquer la fonction sur chaque ligne de turns_df
turns_df["jsonl_line"] = turns_df.apply(turn_to_jsonl_line, axis=1)

# garder seulement les lignes non nulles (tours user)
jsonl_lines = turns_df["jsonl_line"].dropna()

# écrire le fichier pour le fine-tuning
with open("train.jsonl", "w", encoding="utf-8") as f:
    for line in jsonl_lines:
        f.write(line + "\n")


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "LiquidAI/LFM2.5-1.2B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",       # "cpu" si tu n'as pas de GPU
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)


`torch_dtype` is deprecated! Use `dtype` instead!


In [10]:
messages = [
    {
        "role": "system",
        "content": (
            "You are an assistant that extracts flight booking information: "
            "intent, or_city, dst_city, dep_date, ret_date, budget. "
            "Always answer with a valid JSON object and nothing else."
        ),
    },
    {
        "role": "user",
        "content": "Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.",
    },
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)


In [11]:
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.1,
        top_k=50,
        top_p=0.1,
        repetition_penalty=1.05,
    )

generated = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print(generated)


{
  "intent": "book_flight",
  "or_city": "Gotham City",
  "dst_city": "Mos Eisley",
  "dep_date": "2023-11-15",
  "ret_date": "2023-11-20",
  "budget": "2100"
}


In [ ]:
from datasets import load_dataset

ALL_PATH = "all.jsonl"

ds = load_dataset("json", data_files=ALL_PATH, split="train")

# Mélanger
ds = ds.shuffle(seed=42)

# Split 80% train, 20% test
split = ds.train_test_split(test_size=0.2, seed=42)
ds_train = split["train"]
ds_test  = split["test"]

# (Option) garder une petite validation à partir du train
valid_split = ds_train.train_test_split(test_size=0.1, seed=42)
ds_train = valid_split["train"]
ds_val   = valid_split["test"]

# Sauvegarder en JSONL si tu veux des fichiers séparés
ds_train.to_json("train.jsonl", lines=True, orient="records")
ds_val.to_json("val.jsonl", lines=True, orient="records")
ds_test.to_json("test.jsonl", lines=True, orient="records")


Creating json from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 116.54ba/s]


863705

: 

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
)

MODEL_ID   = "LiquidAI/LFM2.5-1.2B-Instruct"
TRAIN_PATH = "train.jsonl"
VAL_PATH   = "val.jsonl"
OUTPUT_DIR = "lfm2_finetuned_flyme"
MAX_LEN    = 512

# 1) Charger datasets
ds_train = load_dataset("json", data_files=TRAIN_PATH, split="train")
ds_val   = load_dataset("json", data_files=VAL_PATH,   split="train")

# 2) Tok + modèle
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
)

# 3) Pré-traitement robuste
def preprocess(example):
    messages = example["messages"]

    # texte complet (system + user + assistant)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    enc = tokenizer(
        text,
        padding="max_length",   # TOUTES les séquences auront MAX_LEN
        truncation=True,
        max_length=MAX_LEN,
    )

    # labels = input_ids, même taille
    enc["labels"] = enc["input_ids"].copy()

    # on ne garde que les colonnes utiles
    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": enc["labels"],
    }

ds_train_tok = ds_train.map(preprocess, batched=False)
ds_val_tok   = ds_val.map(preprocess,   batched=False)

# 4) Trainer
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train_tok,
    eval_dataset=ds_val_tok,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


Generating train split: 7492 examples [00:00, 811563.16 examples/s]
Generating train split: 833 examples [00:00, 152264.24 examples/s]
Map: 100%|██████████| 833/833 [00:00<00:00, 2132.94 examples/s]
C:\Users\Rafik\AppData\Local\Temp\ipykernel_4708\88741086.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
d:\haythem_gassab\projet 10\flyme-chat-widget-test\mock-backend\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [3]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Modèle open (pas de gated repo)
MODEL_ID  = "LiquidAI/LFM2-350M-Extract"  # 350M, spécialisé extraction
TEST_PATH = "test.jsonl"

# 1) Charger modèle + tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
)
model.eval()

# 2) Charger dataset de test
ds_test = load_dataset("json", data_files=TEST_PATH, split="train")

def eval_example(example):
    messages = example["messages"]

    # JSON de référence = dernier message assistant
    ref_str = messages[-1]["content"]
    try:
        ref_json = json.loads(ref_str)
    except Exception:
        return {"exact_match": 0.0, "intent_correct": 0.0}

    # system + user uniquement pour la prédiction
    eval_messages = messages[:-1]

    # Prompt renforcé avec exemple
    system_content = (
        "You are an assistant that extracts flight booking information "
        "from the user's message and returns a JSON object.\n"
        "Fields: intent, or_city, dst_city, dep_date, ret_date, budget.\n\n"
        "Example:\n"
        "User: I want to book a flight from Gotham City to Mos Eisley from November 15th to November 20th with a budget of 2100 dollars.\n"
        "Assistant: {\"intent\": \"book_flight\", \"or_city\": \"Gotham City\", \"dst_city\": \"Mos Eisley\", \"dep_date\": \"2023-11-15\", \"ret_date\": \"2023-11-20\", \"budget\": \"2100\"}\n\n"
        "Now extract the JSON for the following user message."
    )

    if len(eval_messages) > 0 and eval_messages[0]["role"] == "system":
        eval_messages[0]["content"] = system_content
    else:
        eval_messages = [{"role": "system", "content": system_content}] + eval_messages

    prompt = tokenizer.apply_chat_template(
        eval_messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            temperature=0.0,
        )

    gen_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True,
    ).strip()

    # Nettoyer pour ne garder que le JSON
    start = gen_text.find("{")
    end = gen_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        gen_text_clean = gen_text[start:end+1]
    else:
        gen_text_clean = gen_text

    try:
        pred_json = json.loads(gen_text_clean)
    except Exception:
        return {"exact_match": 0.0, "intent_correct": 0.0}

    intent_correct = float(pred_json.get("intent") == ref_json.get("intent"))

    keys = ["intent", "or_city", "dst_city", "dep_date", "ret_date", "budget"]
    exact = float(all(pred_json.get(k) == ref_json.get(k) for k in keys))

    return {"exact_match": exact, "intent_correct": intent_correct}

# 4) Évaluation
ds_eval = ds_test.select(range(200))  # 200 exemples pour aller vite
results = ds_eval.map(eval_example)

exact_match_score = sum(results["exact_match"]) / len(results)
intent_acc        = sum(results["intent_correct"]) / len(results)

print("Intent accuracy :", intent_acc)
print("Exact match     :", exact_match_score)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Map: 100%|██████████| 200/200 [13:52<00:00,  4.16s/ examples]

Intent accuracy : 0.0
Exact match     : 0.0


In [10]:
def eval_example(example):
    messages = example["messages"]

    # JSON de référence
    ref_str = messages[-1]["content"]
    try:
        ref_json = json.loads(ref_str)
    except Exception:
        return {"exact_match": 0.0, "intent_correct": 0.0}

    eval_messages = messages[:-1]

    # System renforcé
    system_content = (
        "You are an assistant that extracts flight booking information "
        "from the user's message and returns a JSON object.\n"
        "Fields: intent, or_city, dst_city, dep_date, ret_date, budget.\n\n"
        "Example:\n"
        "User: I want to book a flight from Gotham City to Mos Eisley from November 15th to November 20th with a budget of 2100 dollars.\n"
        "Assistant: {\"intent\": \"book_flight\", \"or_city\": \"Gotham City\", \"dst_city\": \"Mos Eisley\", \"dep_date\": \"2023-11-15\", \"ret_date\": \"2023-11-20\", \"budget\": \"2100\"}\n\n"
        "Now extract the JSON for the following user message."
    )

    if len(eval_messages) > 0 and eval_messages[0]["role"] == "system":
        eval_messages[0]["content"] = system_content
    else:
        eval_messages = [{"role": "system", "content": system_content}] + eval_messages

    prompt = tokenizer.apply_chat_template(
        eval_messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    
    # DEBUG : afficher le prompt et la phrase
    print("=== DEBUG ===")
    print("User message :", eval_messages[-1]["content"])
    print("Prompt envoyé :", prompt[:200] + "...")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            temperature=0.0,
        )

    gen_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True,
    ).strip()

    # DEBUG : afficher la génération brute
    print("Génération brute :", repr(gen_text))
    print("=================")

    # Nettoyer pour JSON
    start = gen_text.find("{")
    end = gen_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        gen_text_clean = gen_text[start:end+1]
    else:
        gen_text_clean = gen_text

    print("JSON nettoyé :", repr(gen_text_clean))

    try:
        pred_json = json.loads(gen_text_clean)
        print("JSON parsé :", pred_json)
    except Exception as e:
        print("Erreur parsing :", e)
        return {"exact_match": 0.0, "intent_correct": 0.0}

    # Comparaison
    intent_correct = float(pred_json.get("intent") == ref_json.get("intent"))
    keys = ["intent", "or_city", "dst_city", "dep_date", "ret_date", "budget"]
    exact = float(all(pred_json.get(k) == ref_json.get(k) for k in keys))

    print("Ref JSON :", ref_json)
    print("Intent OK :", intent_correct)
    print("Exact match :", exact)
    print("\n")

    return {"exact_match": exact, "intent_correct": intent_correct}


In [11]:
ds_eval = ds_test.select(range(3))  # juste 3 exemples
results = ds_eval.map(eval_example)

exact_match_score = sum(results["exact_match"]) / len(results)
intent_acc        = sum(results["intent_correct"]) / len(results)

print("Intent accuracy :", intent_acc)
print("Exact match     :", exact_match_score)


Intent accuracy : 0.0
Exact match     : 0.0


In [17]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Dict, Any

MODEL_ID = "LiquidAI/LFM2.5-1.2B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")

class SmartFlightAgent:
    def __init__(self):
        self.conversation_history = []
        self.state: Dict[str, Any] = {
            "intent": None,
            "or_city": None,
            "dst_city": None,
            "dep_date": None,
            "ret_date": None,
            "budget": None,
            "status": "waiting_intent"  # waiting_intent, collecting_info, ready_confirm
        }
    
    def chat(self, user_message: str) -> str:
        self.conversation_history.append({"role": "user", "content": user_message})
        
        # Build full context
        response = self._get_agent_response(user_message)
        self.conversation_history.append({"role": "assistant", "content": response})
        
        return response
    
    def _get_agent_response(self, message: str) -> str:
        system_prompt = self._build_system_prompt()
        
        messages = [{"role": "system", "content": system_prompt}] + self.conversation_history[-4:]
        
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.1,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        full_response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
        
        # Parse structured response
        self._parse_and_update(full_response)
        
        return full_response
    
    def _build_system_prompt(self) -> str:
        state_str = json.dumps(self.state, indent=2)
        return f"""FLIGHT BOOKING AGENT v2

Current state:
{state_str}

RULES (IMPORTANT):
1. ONLY flights. "hotel/car" → "I only book flights"
2. Extract ONLY explicit info. NO guessing dates/cities
3. Ask ONE missing field at a time  
4. When complete → Summary + confirm
5. Update state in JSON format first line

RESPONSE FORMAT:
Line 1: JSON state update
Line 2+: Human response

Example:
{{"or_city": "Paris"}}
Perfect! Where to?"""

    def _parse_and_update(self, response: str):
        """Parse JSON from first line, update state"""
        lines = response.strip().split('\n')
        if lines:
            json_line = lines[0].strip()
            try:
                update = json.loads(json_line)
                self.state.update({k: v for k, v in update.items() if v is not None})
            except json.JSONDecodeError:
                pass
    
    def get_status(self) -> str:
        missing = sum(1 for v in [self.state[k] for k in ['or_city','dst_city','dep_date','ret_date','budget']] if not v)
        if missing == 0:
            return "READY_TO_CONFIRM"
        return f"NEEDS_{missing}_MORE_FIELDS"

# =====================================================
# TEST NOUVELLE ROUTINE
# =====================================================
agent = SmartFlightAgent()

print("🧠 SMART AGENT v2 - Stateful LLM")
print("="*60)

test_convo = [
    "I need a hotel",
    "Book flight Paris to NYC $800", 
    "November 15-20",
    "Yes book it"
]

for msg in test_convo:
    print(f"\n👤 YOU: {msg}")
    response = agent.chat(msg)
    print(f"🤖 BOT: {response}")
    print(f"📊 STATUS: {agent.get_status()}")
    print(f"📋 STATE: {agent.state}")

print("\n✅ Smart Agent v2 ready!")


🧠 SMART AGENT v2 - Stateful LLM

👤 YOU: I need a hotel
🤖 BOT: {"or_city": "New York"}

Summary: You are looking for a hotel in New York.  
Confirm?
📊 STATUS: NEEDS_4_MORE_FIELDS
📋 STATE: {'intent': None, 'or_city': 'New York', 'dst_city': None, 'dep_date': None, 'ret_date': None, 'budget': None, 'status': 'waiting_intent'}

👤 YOU: Book flight Paris to NYC $800
🤖 BOT: {"intent": "book_flight", "or_city": "Paris", "dep_date": null, "ret_date": null, "budget": 800, "status": "waiting_intent"}
📊 STATUS: NEEDS_3_MORE_FIELDS
📋 STATE: {'intent': 'book_flight', 'or_city': 'Paris', 'dst_city': None, 'dep_date': None, 'ret_date': None, 'budget': 800, 'status': 'waiting_intent'}

👤 YOU: November 15-20
🤖 BOT: {"intent": "book_flight", "or_city": "Paris", "dep_date": "2023-11-15", "ret_date": "2023-11-20", "budget": 800, "status": "waiting_intent"}
📊 STATUS: NEEDS_1_MORE_FIELDS
📋 STATE: {'intent': 'book_flight', 'or_city': 'Paris', 'dst_city': None, 'dep_date': '2023-11-15', 'ret_date': '2023-11-20